## Exploratory Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
import plotly.express as px
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

In [ ]:
pbj_data = pd.read_csv('PBJ_Daily_Nurse_Staffing_Q2_2024.csv', low_memory= False, encoding = 'latin-1')
# pbj_cols = pbj_data.columns.tolist()
# pbj_cols = [x.lower() for x in pbj_cols]
# pbj_data.columns = pbj_cols
pbj_data.columns = [x.lower() for x in pbj_data.columns.tolist()]
pbj_data.head(10)

In [ ]:
pbj_ratios = pbj_data.copy()
pbj_ratios = pbj_ratios.groupby('provnum').agg(mdscensus_mean = ('mdscensus','mean'),
                                               mdscensus_sum = ('mdscensus','sum'),
                                               hrs_rndon = ('hrs_rndon','sum'),
                                               hrs_rndon_emp = ('hrs_rndon_emp','sum'),
                                               hrs_rdon_ctr = ('hrs_rndon_ctr','sum'),
                                               hrs_rnadmin = ('hrs_rnadmin','sum'),
                                               hrs_rnadmin_emp = ('hrs_rnadmin_emp','sum'),
                                               hrs_rnadmin_ctr = ('hrs_rnadmin_ctr','sum'),
                                               hrs_rn = ('hrs_rn','sum'),
                                               hrs_rn_emp = ('hrs_rn_emp','sum'),
                                               hrs_rn_ctr = ('hrs_rn_ctr','sum'),
                                               hrs_lpnadmin = ('hrs_lpnadmin','sum'),
                                               hrs_lpnadmin_emp = ('hrs_lpnadmin_emp','sum'),
                                               hrs_lpnadmin_ctr = ('hrs_lpnadmin_ctr','sum'),
                                               hrs_lpn = ('hrs_lpn','sum'),
                                               hrs_lpn_emp = ('hrs_lpn_emp','sum'),
                                               hrs_lpn_ctr = ('hrs_lpn_ctr','sum'),
                                               hrs_cna = ('hrs_cna','sum'),
                                               hrs_cna_emp = ('hrs_cna_emp','sum'),
                                               hrs_cna_ctr = ('hrs_cna_ctr','sum'),
                                               hrs_natrn = ('hrs_natrn','sum'),
                                               hrs_natrn_emp = ('hrs_natrn_emp','sum'),
                                               hrs_natrn_ctr = ('hrs_natrn_ctr','sum'),
                                               hrs_medaide = ('hrs_medaide','sum'),
                                               hrs_medaide_emp = ('hrs_medaide_emp','sum'),
                                               hrs_medaide_ctr = ('hrs_medaide_ctr','sum'))
pbj_ratios.head()

In [ ]:
pbj_ratios.insert(0, 'provnum', pbj_ratios.index)
pbj_ratios.reset_index(inplace = True, drop=True)
pbj_ratios.head()

In [ ]:
pbj_ratio_cols = pbj_ratios.columns.tolist()
ratio_indx = [x for x in range(3, len(pbj_ratio_cols), 3)]
hrs_ttl_cols = [pbj_ratio_cols[i] for i in ratio_indx]
hrs_emp_cols = [x for x in pbj_ratios.columns.tolist() if x.find('_emp') >= 0]
hrs_ctr_cols = [x for x in pbj_ratios.columns.tolist() if x.find('_ctr') >= 0]
pbj_ratios['hrs_ttl'] = pbj_ratios[hrs_ttl_cols].sum(axis = 1)
pbj_ratios['hrs_ttl_emp'] = pbj_ratios[hrs_emp_cols].sum(axis = 1)
pbj_ratios['hrs_ttl_ctr'] = pbj_ratios[hrs_ctr_cols].sum(axis = 1)
pbj_ratios.head()

In [ ]:
# # pbj_ratio_sub = pbj_ratios.copy()
# pbj_ratios.fillna(0, inplace = True)
# pbj_ratios = pbj_ratios.loc[pbj_ratios.hrs_ttl_emp != 0].copy()
# pbj_ratios = pbj_ratios.loc[pbj_ratios.hrs_ttl_ctr != 0].copy()
# pbj_ratios.head()

In [ ]:
pbj_ratio_cols = pbj_ratios.columns.tolist()
ratio_indx = [x for x in range(3, len(pbj_ratio_cols), 3)]
for i in range(0,len(ratio_indx)):
    if ratio_indx[i] == ratio_indx[-1]:
        temp_str1 = pbj_ratio_cols[ratio_indx[i]] + '_ratio'
        temp_str2 = pbj_ratio_cols[ratio_indx[i]] + '_census_mean'
        val1 = pbj_ratios[pbj_ratio_cols[ratio_indx[i] + 2]] / pbj_ratios[pbj_ratio_cols[ratio_indx[i] + 1]]
        val2 = pbj_ratios[pbj_ratio_cols[ratio_indx[i]]] / pbj_ratios['mdscensus_sum']
        pbj_ratios[temp_str1] = val1 
        pbj_ratios[temp_str2] = val2
    else:        
        temp_str1 = pbj_ratio_cols[ratio_indx[i]] + '_ratio'
        temp_str2 = pbj_ratio_cols[ratio_indx[i]] + '_census_mean'
        insert_1 = i * 2 + ratio_indx[i] + 3
        insert_2 = i * 2 + ratio_indx[i] + 4
        val1 = pbj_ratios[pbj_ratio_cols[ratio_indx[i] + 2]] / pbj_ratios[pbj_ratio_cols[ratio_indx[i] + 1]]
        val2 = pbj_ratios[pbj_ratio_cols[ratio_indx[i]]] / pbj_ratios['mdscensus_sum']
        pbj_ratios.insert(insert_1, temp_str1, val1)
        pbj_ratios.insert(insert_2, temp_str2, val2)

pbj_ratios.head()

In [ ]:
# hrs_ttl_cols = [pbj_ratio_cols[i] for i in ratio_indx]
# drop_cols = hrs_emp_cols + hrs_ctr_cols + hrs_ttl_cols + ['mdscensus_sum'] # , 'hrs_ttl_emp', 'hrs_ttl_ctr']
# pbj_ratios.drop(drop_cols, axis = 1, inplace = True)
pbj_ratios.replace([np.inf, -np.inf], pd.NA, inplace = True)
pbj_ratios.fillna(0.0, inplace = True)
pbj_ratios.head()

In [ ]:
pbj_hrs_ttls = pbj_ratios[['provnum', 'mdscensus_sum', 'hrs_ttl', 'hrs_ttl_emp', 'hrs_ttl_ctr']].copy()
pbj_hrs_ttls.head()

In [ ]:
pbj_data.describe()

In [ ]:
prov_city = pd.pivot_table(pbj_data, index = ['state', 'county_name', 'city'], values = ['provnum'], aggfunc = 'nunique')
prov_city.head()

In [ ]:
prov_county = pd.pivot_table(pbj_data, index = ['state', 'county_name'], values = ['provnum'], aggfunc = 'nunique')
prov_county.head()

In [ ]:
prov_state = pd.pivot_table(pbj_data, index = 'state', values = ['provnum'], aggfunc = 'nunique')
prov_state.head()

In [ ]:
prov_state.describe()

In [ ]:
patient_qual = pd.read_csv('NH_QualityMsr_MDS_Oct2024.csv', low_memory = False, encoding = 'latin-1')
patient_qual.head(10)

In [ ]:
star_ratings = patient_qual[patient_qual['Used in Quality Measure Five Star Rating'] == 'Y'].copy()

In [ ]:
patient_qual_ttl = patient_qual.groupby('CMS Certification Number (CCN)').agg({})

In [ ]:
num_qual_codes = len(patient_qual['Measure Code'].unique().tolist())
num_qual_codes

In [ ]:
measure_desc_list = patient_qual['Measure Description'].unique().tolist()
measure_desc_list

In [ ]:
state_avgs = pd.read_csv('NH_StateUSAverages_Oct2024.csv', low_memory = False, encoding = 'latin-1')
state_avgs.head(10)

In [ ]:
facility_qual_rates = pd.read_csv('Skilled_Nursing_Facility_Quality_Reporting_Program_Provider_Data_Oct2024.csv', low_memory = False, encoding = 'latin-1')
facility_qual_rates.head(10)

# Provider Information

In [ ]:
provider_info = pd.read_csv('NH_ProviderInfo_Oct2024.csv', low_memory = False, encoding = 'latin-1')
provider_info.head(10)

In [ ]:
test_prov = provider_info[['CMS Certification Number (CCN)', 'State', 'County/Parish', 'City/Town', 'Provider SSA County Code']].copy()

In [ ]:
owner_rating = provider_info.groupby('Ownership Type').agg({'Overall Rating':'mean'})
owner_rating

In [ ]:
owner_staff_score = provider_info.groupby('Ownership Type').agg({'Staffing Rating':'mean'})
owner_staff_score

In [ ]:
sns.barplot(provider_info, x = 'Ownership Type', y = 'Overall Rating')
plt.show()

In [ ]:
sub_ratios = pbj_ratios[['provnum', 'hrs_ttl_ratio', 'hrs_ttl_census_mean']].copy()
sub_info = provider_info[['CMS Certification Number (CCN)', 'Ownership Type', 'Overall Rating', 'Staffing Rating']].copy()
sub_info.columns = ['provnum', 'Ownership Type', 'Overall Rating', 'Staffing Rating']
sub_scores = pd.merge(sub_ratios, sub_info, how = 'left', on = 'provnum')
sub_scores = sub_scores[['provnum', 'Ownership Type', 'Overall Rating', 'Staffing Rating', 'hrs_ttl_census_mean', 'hrs_ttl_ratio']].copy()
sub_scores.sort_values(['Ownership Type', 'Overall Rating', 'Staffing Rating', 'hrs_ttl_census_mean', 'hrs_ttl_ratio'],
                    #    ascending = [False, False, False, False, False], 
                       inplace = True)
sub_scores.head(20)

In [ ]:
prov_score_cols = ['CMS Certification Number (CCN)', 'Overall Rating', 'Staffing Rating'] + provider_info.columns.tolist()[-13:-6]
prov_score = provider_info[prov_score_cols].copy()
prov_score.columns = ['provnum'] + prov_score_cols[1:]
prov_score.head()

In [ ]:
pbj_ratios_score = pd.merge(pbj_ratios, prov_score, how = 'left', on = 'provnum')
pbj_ratios_score.head(10)

In [ ]:
pbj_sc_cols = pbj_ratios_score.columns.tolist()
pbj_ratios_score.fillna(0, inplace = True)
for i in range(1,len(pbj_sc_cols)):
    pbj_ratios_score[pbj_sc_cols[i]] = pbj_ratios_score[pbj_sc_cols[i]].astype('float')
pbj_ratios_score.drop('provnum', axis = 1, inplace = True)
pbj_ratios_score.head()

In [ ]:
pbj_rs_corr = pbj_ratios_score.corr()
pbj_rs_corr

In [ ]:
fig, ax = plt.subplots(figsize = (100, 100))
sns.heatmap(pbj_rs_corr, annot = True, cmap = 'coolwarm')
plt.show()

In [ ]:
scaler = StandardScaler()
ratios_scaled = pd.DataFrame(scaler.fit_transform(pbj_ratios_score), columns=pbj_ratios_score.columns)
ratios_scaled.head(10)

In [ ]:
ratios_corr = ratios_scaled.corr()
ratios_corr

In [ ]:
fig, ax = plt.subplots(figsize = (100, 100))
sns.heatmap(ratios_corr, annot = True, cmap = 'coolwarm')
plt.show()

In [ ]:
prov_info_cols = provider_info.columns.tolist()
prov_examp = provider_info.iloc[1].values.tolist()
prov_exampl_w_cols = []
for i in range(len(prov_info_cols)):
    temp = (i, prov_info_cols[i], prov_examp[i])
    prov_exampl_w_cols.append(temp)
prov_exampl_w_cols

In [ ]:
ownership_type_list = provider_info['Ownership Type'].unique().tolist()
provider_info.insert(10, 'Ownership Type Code', pd.NA)
provider_info['Ownership Type Code'] = provider_info[['Ownership Type']].apply(lambda x: ownership_type_list.index(str(x['Ownership Type'])), axis = 1)

In [ ]:
provider_type_list = provider_info['Provider Type'].unique().tolist()
provider_info.insert(15, 'Provider Type Code', pd.NA)
provider_info['Provider Type Code'] = provider_info[['Provider Type']].apply(lambda x: provider_type_list.index(str(x['Provider Type'])), axis = 1)

In [ ]:
def special_focus_status(sff):
    match sff:
        case 'SFF Candidate':
            return 1
        case 'SFF':
            return 2
        case _:
            return 0

provider_info.insert(23, 'SFF Code', pd.NA)
provider_info['SFF Code'] = provider_info[['Special Focus Status']].apply(lambda x: special_focus_status(x['Special Focus Status']), axis = 1)

In [ ]:
def family_council(famCon):
    match famCon:
        case 'Resident':
            return 1
        case 'Family':
            return 2
        case 'Both':
            return 3
        case _:
            return 0
        
provider_info.insert(28, 'With Council Code', pd.NA)
provider_info['With Council Code'] = provider_info[['With a Resident and Family Council']].apply(lambda x: family_council(x['With a Resident and Family Council']), axis = 1)

In [ ]:
provider_info.replace('Y', 1, inplace = True)
provider_info.replace('N', 0, inplace = True)
provider_info.replace('Yes', 1, inplace = True)
provider_info.replace('No', 0, inplace = True)

In [ ]:
col_nums = [x for x in range(10,14)] + [v for v in range(15,17)] + [21] + [y for y in range(23,27)] + [z for z in range(28,70)] + [u for u in range(71,78)] + [w for w in range(94,102)]
col_names = [prov_info_cols[i] for i in col_nums]
col_names = [x for x in col_names if x.find('Footnote') < 0]
sub_prov_info = provider_info[col_names].copy()
sub_prov_info.head()

In [ ]:
import numpy as np
sub_prov_info.replace('Partial', 2, inplace = True)
sub_prov_info.astype(np.float64())

In [ ]:
scaler = StandardScaler()
sub_scaled = pd.DataFrame(scaler.fit_transform(sub_prov_info), columns=sub_prov_info.columns)
sub_scaled.head(10)

In [ ]:
score_corr = sub_scaled.corr()
score_corr

In [ ]:
flatten_corr = score_corr.stack().reset_index()
flatten_corr.columns = ['Variable 1', 'Variable 2', 'Correlation Score']
flatten_corr = flatten_corr[flatten_corr['Variable 1'] != flatten_corr['Variable 2']]
flatten_corr.sort_values(by = 'Correlation Score', ascending = False, inplace = True)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

data = score_corr.copy()
data.columns = [x for x in range(len(data.columns))]
sns.heatmap(data, annot = True, cmap = 'coolwarm')
plt.show()

In [ ]:
corr_nums = sub_scaled.copy()
corr_nums.columns = [x for x in range(len(corr_nums.columns))]
corr_nums = corr_nums.corr()

In [ ]:
fig, ax = plt.subplots(figsize = (200, 200))
sns.heatmap(corr_nums, annot = True, cmap = 'coolwarm')
plt.show()

In [ ]:
flat_corr = flatten_corr[flatten_corr['Correlation Score'] < 0.999999].copy()

def sort_the_values(str1, str2):
    temp = [str1, str2]
    temp.sort()
    return temp

flat_corr['Sort Column'] = flat_corr[['Variable 1', 'Variable 2']].apply(lambda x: '|'.join(sort_the_values(x['Variable 1'], x['Variable 2'])), axis = 1)
flat_corr.drop_duplicates('Sort Column', inplace = True)
flat_corr.drop('Sort Column', axis = 1, inplace = True)
# flat_corr.head(50)

In [ ]:
sub_flat_corr = flat_corr.loc[(flat_corr['Variable 1'].str.lower().str.find('hours') > 0) & (flat_corr['Variable 2'].str.lower().str.find('hours') < 0)]
sub_flat_corr.head(20)

In [ ]:
sub_flat_corr.tail(20)

In [ ]:
cut_points = pd.read_csv('NH_HlthInspecCutpointsState_Oct2024.csv', low_memory = False, encoding = 'latin-1')
cut_points

In [ ]:
pbj_hrs_ttls.head()

In [ ]:
provider_sub = provider_info[['CMS Certification Number (CCN)', 'Ownership Type', 'State', 'County/Parish', 'Staffing Rating', 'Overall Rating']].copy()
provider_sub.rename({'CMS Certification Number (CCN)':'provnum'}, axis = 1, inplace = True)
provider_sub.head()

# Provider Aggregate

In [ ]:
provider_agg = pd.merge(provider_sub, pbj_hrs_ttls, how = 'left', on = 'provnum')
provider_agg[['Type', 'SubType']] = provider_agg['Ownership Type'].str.split(' - ', expand = True)
provider_agg.drop('Ownership Type', axis = 1, inplace = True)
provider_agg.rename({'Type':'Ownership Type', 'SubType':'Ownership SubType'}, inplace = True, axis = 1)
provider_agg.head()

## Ownership Type Averages

In [ ]:
owner_type_avgs = provider_agg.groupby('Ownership Type').agg(staff_rating = ('Staffing Rating', 'mean'),
                                                             overall_rating = ('Overall Rating', 'mean'),
                                                             census_sum = ('mdscensus_sum', 'sum'),
                                                             hrs_ttl = ('hrs_ttl', 'sum'),
                                                             hrs_ttl_emp = ('hrs_ttl_emp', 'sum'),
                                                             hrs_ttl_ctr = ('hrs_ttl_ctr', 'sum'))
owner_type_avgs.insert(0, 'Ownership Type', owner_type_avgs.index)
owner_type_avgs.reset_index(drop = True, inplace = True)
owner_type_avgs

In [ ]:
owner_type_avgs['hrs_per_resident'] = owner_type_avgs.hrs_ttl / owner_type_avgs.census_sum
owner_type_avgs['ctr_emp_ratio'] = owner_type_avgs.hrs_ttl_ctr / owner_type_avgs.hrs_ttl_emp
owner_type_avgs

In [ ]:
import plotly.graph_objects as go
for_profit_df = owner_type_avgs[owner_type_avgs['Ownership Type'] == 'For profit']
non_profit_df = owner_type_avgs[owner_type_avgs['Ownership Type'] == 'Non profit']
government_df = owner_type_avgs[owner_type_avgs['Ownership Type'] == 'Government']
staff_score_vs_hrs_per_resident = go.Figure(
        [go.Scatter(x = for_profit_df['hrs_per_resident'],
                    y = for_profit_df['staff_rating'],
                    mode = 'markers',
                    name = 'For Profit',
                    marker = dict(size = for_profit_df['overall_rating'].values[0] * 10),
                    hovertemplate = '<b>For Profit</b><br>Avg Hrs per Resident: %{x:.2f}<br>Overall Rating: %{y:.2f}'
                    ),
        go.Scatter(x = non_profit_df['hrs_per_resident'],
                    y = non_profit_df['staff_rating'],
                    mode = 'markers',
                    name = 'Non Profit',
                    marker = dict(size = non_profit_df['overall_rating'].values[0] * 10),
                    hovertemplate = '<b>Non Profit</b><br>Avg Hrs per Resident: %{x:.2f}<br>Overall Rating: %{y:.2f}'
                    ),
        go.Scatter(x = government_df['hrs_per_resident'],
                    y = government_df['staff_rating'],
                    mode = 'markers',
                    name = 'Government',
                    marker = dict(size = government_df['overall_rating'].values[0] * 10),
                    hovertemplate = '<b>Government</b><br>Avg Hrs per Resident: %{x:.2f}<br>Overall Rating: %{y:.2f}'
                    )
        ],
    layout = dict(
        title = dict(
            text = 'Staff Rating vs Average Total Daily Hours per Resident',
            subtitle = dict(text = 'with Average Overall Rating as size')
        ),
        xaxis_title = 'Average Total Daily Hours per Resident',
        yaxis_title = 'Average Staff Rating'
    )
)
staff_score_vs_hrs_per_resident.update_layout(title = {'x': 0.5, 'xanchor': 'center'})
staff_score_vs_hrs_per_resident.show()

In [ ]:
ctr_ratio_vs_hrs_per_resident = go.Figure(
        [go.Scatter(x = for_profit_df['hrs_per_resident'],
                    y = for_profit_df['overall_rating'],
                    mode = 'markers',
                    name = 'For Profit',
                    marker = dict(size = for_profit_df['ctr_emp_ratio'].values[0] * 500),
                    hovertemplate = '<b>Government</b><br>Avg Hrs per Resident: %{x:.2f}' +
                    '<br>Contractor to Employee Hours Ratio: {0:.4f}'.format(for_profit_df['ctr_emp_ratio'].values[0]) +
                    '<br>Overall Rating: %{y:.2f}'
                    ),
        go.Scatter(x = non_profit_df['hrs_per_resident'],
                    y = non_profit_df['overall_rating'],
                    mode = 'markers',
                    name = 'Non Profit',
                    marker = dict(size = non_profit_df['ctr_emp_ratio'].values[0] * 500),
                    hovertemplate = '<b>Government</b><br>Avg Hrs per Resident: %{x:.2f}' +
                    '<br>Contractor to Employee Hours Ratio: {0:.4f}'.format(non_profit_df['ctr_emp_ratio'].values[0]) +
                    '<br>Overall Rating: %{y:.2f}'
                    ),
        go.Scatter(x = government_df['hrs_per_resident'],
                    y = government_df['overall_rating'],
                    mode = 'markers',
                    name = 'Government',
                    marker = dict(size = government_df['ctr_emp_ratio'].values[0] * 500),
                    hovertemplate = '<b>Government</b><br>Avg Hrs per Resident: %{x:.2f}' +
                    '<br>Contractor to Employee Hours Ratio: {0:.4f}'.format(government_df['ctr_emp_ratio'].values[0]) +
                    '<br>Overall Rating: %{y:.2f}'
                    )
        ],
    layout = dict(
        title = dict(
            text = 'Staff Rating vs Average Total Daily Hours per Resident',
            subtitle = dict(text = 'with Average Overall Rating as size')
        ),
        xaxis_title = 'Average Total Daily Hours per Resident',
        yaxis_title = 'Contractor to Employee Hours Ratio'
    )
)
ctr_ratio_vs_hrs_per_resident.update_layout(title = {'x': 0.5, 'xanchor': 'center'})
ctr_ratio_vs_hrs_per_resident.show()

## State Averages

In [ ]:
state_avgs = provider_agg.groupby('State').agg(staff_rating = ('Staffing Rating', 'mean'),
                                               overall_rating = ('Overall Rating', 'mean'),
                                               census_sum = ('mdscensus_sum', 'sum'),
                                               hrs_ttl = ('hrs_ttl', 'sum'),
                                               hrs_ttl_emp = ('hrs_ttl_emp', 'sum'),
                                               hrs_ttl_ctr = ('hrs_ttl_ctr', 'sum'))
state_avgs.insert(0, 'State', state_avgs.index)
state_avgs.reset_index(drop = True, inplace = True)
state_avgs.dropna(inplace = True)
state_avgs = state_avgs[state_avgs['State'] != 'PR'].copy()
state_avgs['hrs_per_resident'] = state_avgs.hrs_ttl / state_avgs.census_sum
state_avgs['ctr_emp_ratio'] = state_avgs.hrs_ttl_ctr / state_avgs.hrs_ttl_emp
# for col in state_avgs.columns:
#     state_avgs[col] = state_avgs[col].astype('str')
state_avgs['text'] = state_avgs[['State', 'hrs_per_resident', 'overall_rating', 'staff_rating', 'ctr_emp_ratio']].apply(lambda x: \
    (x['State'] + '<br>' +
    'Avg Total Daily Hours per Resident: {:.2f}'.format(x['hrs_per_resident']) + '<br>' +
    'Average Overall Rating: {:.2f}'.format(x['overall_rating']) + '<br>' +
    'Average Staff Rating: {:.2f}'.format(x['staff_rating']) + '<br>' +
    'Contractor to Employee Hour Ratio: {:.4f}'.format(x['ctr_emp_ratio'])), axis = 1)
state_avgs

In [ ]:
us_by_avg_hrs_resd = go.Figure(
    go.Choropleth(
        locations = state_avgs['State'],
        z = state_avgs['hrs_per_resident'],
        locationmode = 'USA-states',
        colorscale = 'Viridis',
        colorbar_title = 'Average Total<br>Daily Hours<br>per Resident'
    )
)
us_by_avg_hrs_resd.update_layout(
    title_text = 'Average Total Daily Hours per Resident by State',
    geo = {
        'scope':'usa',
        'projection': go.layout.geo.Projection(type = 'albers usa'),
        'showlakes': True,
        'lakecolor':'rgb(255, 255, 255)'
    }
)
us_by_avg_hrs_resd.show()

In [ ]:
df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')
df_sample.head()

In [ ]:
df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
fips_codes = df_sample[['County FIPS Code', 'County Name/State Abbreviation']].copy()
fips_codes[['County/Parish', 'State']] = fips_codes['County Name/State Abbreviation'].str.split(', ', expand = True)
fips_codes['County/Parish'] = fips_codes['County/Parish'].apply(lambda x: str(x).lower())
# fips_codes.rename({})
test_prov['County/Parish'] = test_prov['County/Parish'].apply(lambda x: str(x).lower())
test_prov['Provider SSA County Code'] = test_prov['Provider SSA County Code'].astype('str')
# test_prov.rename({})
cnty_code_test = pd.merge(fips_codes, test_prov, how = 'left', on = ['State', 'County/Parish'])
# cnty_code_test = cnty_code_test[cnty_code_test['City/Town'].notnull()]
cnty_code_test.rename({'CMS Certification Number (CCN)':'provnum', 'County FIPS Code':'fips'}, axis = 1, inplace = True)
cnty_codes = cnty_code_test[['provnum', 'fips']].copy()
# cnty_codes.head(20)

In [ ]:
prov_agg_fips = pd.merge(provider_agg, cnty_codes, how = 'left', on = 'provnum')
prov_agg_fips.head()

# Facilities with Average Hours Per Resident Below State Average

Also have below average scores for: Staffing Rating, Overall Rating, and Average Hours per Resident

In [ ]:
state_avg_sub = state_avgs[['State', 'staff_rating', 'overall_rating', 'hrs_per_resident']].copy()
state_prov_agg = provider_agg.copy()
state_prov_agg['fac_hrs_per_res'] = state_prov_agg['hrs_ttl'] / state_prov_agg['mdscensus_sum']
state_prov_avgs = pd.merge(state_prov_agg, state_avg_sub, how = 'left', on = 'State')
state_prov_avgs.head()

In [ ]:
state_prov_avgs['staff_rate_diff'] = state_prov_avgs['Staffing Rating'] - state_prov_avgs['staff_rating']
state_prov_avgs['overall_rate_diff'] = state_prov_avgs['Overall Rating'] - state_prov_avgs['overall_rating']
state_prov_avgs['avg_hrs_diff'] = state_prov_avgs['fac_hrs_per_res'] - state_prov_avgs['hrs_per_resident']
state_prov_avgs.head()

In [ ]:
# Removing any facilities that have 0 Contractor hours
# Removing any facilities that have above average scores for: Staffing Rating, Overall Rating, and Average Hours per Resident
state_prov_avgs_sub = state_prov_avgs.loc[(state_prov_avgs['hrs_ttl_ctr'] > 0) &
                                          (state_prov_avgs['staff_rate_diff'] < 0) &
                                          (state_prov_avgs['overall_rate_diff'] < 0) &
                                          (state_prov_avgs['avg_hrs_diff'] < 0)].copy()
state_prov_avgs_sub.head()

In [ ]:
state_prov_avgs_sub['cnty_cnt'] = state_prov_avgs_sub.groupby(['State', 'County/Parish'])['provnum'].transform('nunique')
state_prov_avgs_sub['state_cnt'] = state_prov_avgs_sub.groupby(['State'])['provnum'].transform('nunique')
state_prov_avgs_sub['cnty_rank'] = state_prov_avgs_sub.sort_values(['avg_hrs_diff', 'overall_rate_diff', 'staff_rate_diff']) \
                                                                   .groupby(['State', 'County/Parish']).cumcount() + 1
state_prov_avgs_sub['state_rank'] = state_prov_avgs_sub.sort_values(['avg_hrs_diff', 'overall_rate_diff', 'staff_rate_diff']) \
                                                                    .groupby(['State']).cumcount() + 1
state_prov_avgs_sub.head()

In [ ]:
prov_names = provider_info[['CMS Certification Number (CCN)', 'Provider Name']].copy()
prov_names.columns = ['provnum', 'prov_name']
prov_names['prov_name'] = prov_names['prov_name'].str.title()
prov_names.head()

In [ ]:
state_prov_avgs_sub = pd.merge(state_prov_avgs_sub, prov_names, how = 'left', on = 'provnum')
state_prov_avgs_sub.head()

In [ ]:
states = state_prov_avgs_sub['State'].unique().tolist()
state_text_df = pd.DataFrame()
for state in states:
    df = state_prov_avgs_sub[(state_prov_avgs_sub['State'] == state) &
                             (state_prov_avgs_sub['state_rank'].isin([1,2,3,4,5]))].copy()
    df['state_text'] = df[['State', 'overall_rating', 'staff_rating', 'hrs_per_resident']].apply(lambda x: \
        ('<b>' + x['State'] + '</b><br>' + 'Avg Hours per Resident({:.2f})'.format(x['hrs_per_resident']) + ' ' +
        'Avg Overall Rating({:.2f})'.format(x['overall_rating']) + ' ' +
        'Avg Staff Rating({:.2f})'.format(x['staff_rating'])), axis = 1)
    df['prov_text'] = df[['prov_name', 'Overall Rating', 'Staffing Rating', 'fac_hrs_per_res']].apply(lambda x: \
        ('<b>' + x['prov_name'] + '</b><br>' + 'Avg Hours per Resident({:.2f})'.format(x['fac_hrs_per_res']) + ' ' +
        'Avg Overall Rating({:.2f})'.format(x['Overall Rating']) + ' ' +
        'Avg Staff Rating({:.2f})'.format(x['Staffing Rating'])), axis = 1)
    state_text = df['state_text'].unique().tolist() + df['prov_text'].tolist()
    temp_df = pd.DataFrame({'State': [state], 'state_text':['<br>'.join(state_text)]})
    state_text_df = pd.concat([state_text_df, temp_df])
state_text_df.head()

In [ ]:
county_dict = {}
county_text_df = pd.DataFrame()
for state in states:
    df = state_prov_avgs_sub[state_prov_avgs_sub['State'] == state].copy()
    cnty_lst = df['County/Parish'].unique().tolist()
    county_dict[state] = cnty_lst 
for key in county_dict.keys():
    for val in county_dict.get(key): 
        # print(key, val)
        df2 = state_prov_avgs_sub[(state_prov_avgs_sub['State'] == key) &
                                 (state_prov_avgs_sub['County/Parish'] == val)].copy()
        # print(len(df2))
        df2['state_text'] = df2[['State', 'overall_rating', 'staff_rating', 'hrs_per_resident']].apply(lambda x: \
            ('<b>' + x['State'] + '</b><br>' + 'Avg Hours per Resident({:.2f})'.format(x['hrs_per_resident']) + ' ' +
            'Avg Overall Rating({:.2f})'.format(x['overall_rating']) + ' ' +
            'Avg Staff Rating({:.2f})'.format(x['staff_rating'])), axis = 1)
        df2['prov_text'] = df2[['prov_name', 'Overall Rating', 'Staffing Rating', 'fac_hrs_per_res']].apply(lambda x: \
            ('<b>' + x['prov_name'] + '</b><br>' + 'Avg Hours per Resident({:.2f})'.format(x['fac_hrs_per_res']) + ' ' +
            'Avg Overall Rating({:.2f})'.format(x['Overall Rating']) + ' ' +
            'Avg Staff Rating({:.2f})'.format(x['Staffing Rating'])), axis = 1)
        state_text = df2['state_text'].unique().tolist() + df2['prov_text'].tolist()
        temp_df = pd.DataFrame({'State': [key], 'County/Parish': [val], 'cnty_text':['<br>'.join(state_text)]})
        county_text_df = pd.concat([county_text_df, temp_df])
county_text_df.head()

In [ ]:
state_prov_avgs_text = pd.merge(state_prov_avgs_sub, state_text_df, how = 'left', on = 'State')
state_prov_avgs_text = pd.merge(state_prov_avgs_text, county_text_df, how = 'left', on = ['State', 'County/Parish'])
state_prov_avgs_text.head()